In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from varname import nameof

In [18]:
#Here we take the naive choice data and make it into Pd Data Frames

NCmetadata = pd.read_csv('220211_2M1F_NaiveChoice_metadata.csv')
X210225_Lane1 = pd.read_csv('220211_2M1F_NaiveChoice_X210225_Lane1_append.csv')
X210225_Lane2 = pd.read_csv('220211_2M1F_NaiveChoice_X210225_Lane2_append.csv')
X210225_Lane3 = pd.read_csv('220211_2M1F_NaiveChoice_X210225_Lane3_append.csv')
X210225_Lane4 = pd.read_csv('220211_2M1F_NaiveChoice_X210225_Lane4_append.csv')
X210226_Lane1 = pd.read_csv('220211_2M1F_NaiveChoice_X210226_Lane1_append.csv')
X210226_Lane2 = pd.read_csv('220211_2M1F_NaiveChoice_X210226_Lane2_append.csv')
X210226_Lane4 = pd.read_csv('220211_2M1F_NaiveChoice_X210226_Lane4_append.csv')
X210318_Lane1 = pd.read_csv('220211_2M1F_NaiveChoice_X210318_Lane1_append.csv')
X210318_Lane2 = pd.read_csv('220211_2M1F_NaiveChoice_X210318_Lane2_append.csv')
X210318_Lane4 = pd.read_csv('220211_2M1F_NaiveChoice_X210318_Lane4_append.csv')
X210401_Lane1 = pd.read_csv('220211_2M1F_NaiveChoice_X210401_Lane1_append.csv')
X210401_Lane2 = pd.read_csv('220211_2M1F_NaiveChoice_X210401_Lane2_append.csv')
X210401_Lane4 = pd.read_csv('220211_2M1F_NaiveChoice_X210401_Lane4_append.csv')
X210824_Lane2 = pd.read_csv('220211_2M1F_NaiveChoice_X210824_Lane2_append.csv')
X210824_Lane3 = pd.read_csv('220211_2M1F_NaiveChoice_X210824_Lane3_append.csv')
X210824_Lane4 = pd.read_csv('220211_2M1F_NaiveChoice_X210824_Lane4_append.csv')

naive_data = [X210225_Lane1,
              X210225_Lane2,
              X210225_Lane3,
              X210225_Lane4,
              X210226_Lane1,
              X210226_Lane2,
              X210226_Lane4,
              X210318_Lane1,
              X210318_Lane2,
              X210318_Lane4,
              X210401_Lane1,
              X210401_Lane2,
              X210401_Lane4,
              X210824_Lane2,
              X210824_Lane3,
              X210824_Lane4,
             ]

#In order to read this, I needed to download and move the csv file into the same folder as my Github projects
#and then I could reference it by name


Lane = ['Lane1', 'Lane2', 'Lane3', 'Lane4', 'Lane1', 'Lane2', 'Lane4','Lane1','Lane2','Lane4','Lane1','Lane2','Lane4', 'Lane2','Lane3','Lane4']



nameof(X210824_Lane4)

'X210824_Lane4'

In [28]:
#You need to extract the Lane information

def extract_name(set_of_datasets, name_of_dataset):
    name_of_dataset = []
    for i in range(0, len(set_of_datasets),1):
        name_of_dataset.append(nameof(set_of_datasets[i]))
    return name_of_dataset
          

ImproperUseError: Argument Call(func=Name(id='str', ctx=Load()), args=[Subscript(value=Name(id='naive_data', ctx=Load()), slice=Constant(value=1), ctx=Load())], keywords=[]) is not a variable or an attribute.

In [ ]:
def function_time_spent_center(dataset):
    time_spent_center = []
    for i in range (0, len(dataset),1):
        row = dataset.loc[i]
        timespentcenter = row.Center_stop-row.Center_start
        time_spent_center.append(timespentcenter)
    total_time_center = np.nansum(time_spent_center)
    return total_time_center

def function_time_spent_right(dataset):
    time_spent_right = []
    for i in range(0, len(dataset),1):
        row = dataset.loc[i]
        timespentright = row.Right_stop - row.Right_start
        time_spent_right.append(timespentright)
    total_time_right = np.nansum(time_spent_right)
    return total_time_right

def function_time_spent_left(dataset):
    time_spent_left = []
    for i in range(0, len(dataset),1):
        row = dataset.loc[i]
        timespentleft = row.Left_stop - row.Left_start
        time_spent_left.append(timespentleft)
    total_time_left = np.nansum(time_spent_left)
    return total_time_left

def function_time_contact_right(dataset):
    time_contact_right = []
    for i in range(0, len(dataset),1):
        row = dataset.loc[i]
        timecontactright = row.Contact_Right_stop - row.Contact_Right_start
        time_contact_right.append(timecontactright)
    total_contact_right = np.nansum(time_contact_right)
    return total_contact_right
    
def function_time_contact_left(dataset):
    time_contact_left = []
    for i in range(0, len(dataset),1):
        row = dataset.loc[i]
        timecontactleft = row.Contact_Left_stop - row.Contact_Left_start
        time_contact_left.append(timecontactleft)
    total_contact_left = np.nansum(time_contact_left)
    return total_contact_left










#I have decided that huddling is unbroken time in contact greater than 20 seconds
def function_time_huddling_right(dataset):
    time_huddling_right = []
    for i in range (0, len(dataset),1):
        row = dataset.loc[i]
        timecontactright = row.Contact_Right_stop - row.Contact_Right_start
        if timecontactright >= 20:
            time_huddling_right.append(timecontactright)
    return time_huddling_right

def function_time_huddling_left(dataset):
    time_huddling_left = []
    for i in range (0, len(dataset),1):
        row = dataset.loc[i]
        timecontactleft = row.Contact_Left_stop - row.Contact_Left_start
        if timecontactleft >= 20:
            time_huddling_left.append(timecontactleft)
    return time_huddling_left

In [ ]:
t_huddle_left = []
t_huddle_right = []
huddle_instances_left = []
huddle_instances_right = []

for i in range (0, len(naive_data),1):
    t_huddle_left.append(np.nansum(function_time_huddling_left(naive_data[i])))
    t_huddle_right.append(np.nansum(function_time_huddling_right(naive_data[i])))
    huddle_instances_left.append(len(function_time_huddling_left(naive_data[i])))
    huddle_instances_right.append(len(function_time_huddling_right(naive_data[i])))

In [ ]:
huddling_time_data = pd.DataFrame()

huddling_time_data.insert(loc = 0,
                      column = 'Lane',
                      value = Lane)
huddling_time_data.insert(loc = 1,
                      column = 'Huddling_time_L',
                      value = t_huddle_left)
huddling_time_data.insert(loc = 2,
                      column = 'Huddling_instances_L',
                      value = huddle_instances_left)
huddling_time_data.insert(loc = 3,
                      column = 'Huddling_time_R',
                      value = t_huddle_right)
huddling_time_data.insert(loc = 4,
                      column = 'Huddling_instances_R',
                      value = huddle_instances_right)

huddling_time_data

In [ ]:
#Winner versus loser? Left up (+), Right Down (-)

Winnervalue = []
for i in range (0, len(huddling_time_data),1):
    left = huddling_time_data.Huddling_time_L[i]
    right = huddling_time_data.Huddling_time_R[i]
    difference = left - right
    total = left + right
    ratio = difference/total
    Winnervalue.append(ratio)
    
Winnervalue

In [ ]:
huddling_time_data.insert(loc = 5,
                      column = 'Preference_ratio',
                      value = Winnervalue)

huddling_time_data

In [ ]:
#Let's make a function to plot huddling data




sns.set_theme()
sns.relplot(
    data= huddling_time_data, kind = "scatter",
    x="Lane", y="Preference_ratio", #col="time",
    #hue="Left", #style="Left", #size="size",
)